## Testing IBMi + Jupyter Notebooks

In [ ]:
import pyodbc, getpass
import pandas as pd
from ipywidgets import interact, interactive, widgets
from IPython.display import display

HOST_OPTS = ['BOX1', 'BOX2', 'PUB400.COM']


# Get user credentials
# Usage: execute cell once, enter credentials, move to next cell

w_host = widgets.Dropdown(options=HOST_OPTS, value=HOST_OPTS[0], description='Host: ')
display(w_host)

w_user = widgets.Text(placeholder='Enter user', description='User: ')
display(w_user)

w_pwd = widgets.Password(placeholder='Enter password', description='Password: ')
display(w_pwd)


# TODO: radio button toggle fixed vs manual hosts?

In [ ]:
# TODO: sliders / filters on query ?
SQL = ''


In [ ]:
host=w_host.value
user=w_user.value
pwd=w_pwd.value

print('Connecting to {} as {} ...'.format(host, user))

conn = pyodbc.connect(driver='{IBM i Access ODBC Driver}', system=host, uid=user, pwd=pwd)

try:
    print('Executing\n  ' + SQL)
    df = pd.read_sql(SQL, conn)
    display(df)
except Exception as e:
    print('Error occurred with DB2 query\n  ' + str(e))
finally:
    conn.close()
    